# Module 5: The Art of Feature Engineering

**Goal:** Create features that capture what customers are actually doing

**Time:** ~20 minutes

**What you'll do:**
1. Create behavioral features from raw events
2. Engineer ratio and change features
3. Add temporal features safely (no leakage!)
4. Measure impact on model performance

---

## Setup

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

# Load data
try:
    df = pd.read_csv('https://raw.githubusercontent.com/189investmentai/ml-foundations-interactive/main/streamcart_customers.csv')
except:
    df = pd.read_csv('../data/streamcart_customers.csv')

print(f"Loaded {len(df):,} customers")
df.head(3)

---

## Part 1: Start with Baseline

First, let's establish performance with raw features only.

In [ ]:
# Raw features (what we've been using)
raw_features = ['tenure_months', 'logins_last_7d', 'logins_last_30d',
                'support_tickets_last_30d', 'items_skipped_last_3_boxes', 'nps_score']

X_raw = df[raw_features].fillna(-1)
y = df['churn_30d']

X_train_raw, X_test_raw, y_train, y_test = train_test_split(X_raw, y, test_size=0.2, random_state=42)

# Train baseline model
baseline_model = lgb.LGBMClassifier(n_estimators=100, max_depth=5, random_state=42, verbose=-1)
baseline_model.fit(X_train_raw, y_train)

baseline_auc = roc_auc_score(y_test, baseline_model.predict_proba(X_test_raw)[:, 1])
print(f"Baseline AUC (raw features): {baseline_auc:.3f}")

---

## Part 2: Create Ratio Features

Ratios often capture behavior better than raw counts.

**Example:** "3 orders" means different things for a 1-month vs 24-month customer.

In [ ]:
# Create a copy for feature engineering
df_eng = df.copy()

# TODO: Create these ratio features
# 
# 1. orders_per_month: total orders divided by tenure
# 2. support_ticket_rate: tickets per month of tenure
# 3. login_consistency: logins_7d / logins_30d (recent vs month)

# Handle division by zero with small epsilon
eps = 0.1

df_eng['orders_per_month'] = None  # Replace with your code
df_eng['support_ticket_rate'] = None  # Replace with your code
df_eng['login_consistency'] = None  # Replace with your code

# Uncomment when ready:
# df_eng['orders_per_month'] = df['orders_total'] / (df['tenure_months'] + eps)
# df_eng['support_ticket_rate'] = df['support_tickets_total'] / (df['tenure_months'] + eps)
# df_eng['login_consistency'] = df['logins_last_7d'] / (df['logins_last_30d'] + eps)

In [ ]:
# ============================================
# SELF-CHECK
# ============================================

assert df_eng['orders_per_month'].notna().all(), "orders_per_month should have no NaN"
assert (df_eng['login_consistency'] >= 0).all(), "login_consistency should be >= 0"

print("✓ Ratio features created!")
print(f"\norders_per_month range: [{df_eng['orders_per_month'].min():.2f}, {df_eng['orders_per_month'].max():.2f}]")
print(f"login_consistency range: [{df_eng['login_consistency'].min():.2f}, {df_eng['login_consistency'].max():.2f}]")

---

## Part 3: Create Change Features

Is the customer getting more or less engaged over time?

**Idea:** Compare recent activity to historical average.

In [ ]:
# TODO: Create change features
#
# 1. login_trend: Are recent logins (7d) above or below their average?
#    Formula: logins_7d - (logins_per_month_avg * 7/30)
#
# 2. recent_activity_drop: Binary flag for low recent activity
#    Formula: 1 if logins_7d == 0 AND logins_30d > 0, else 0

# Expected weekly logins based on monthly average
df_eng['expected_weekly_logins'] = df['logins_per_month_avg'] * (7/30)

df_eng['login_trend'] = None  # Replace with your code
df_eng['recent_activity_drop'] = None  # Replace with your code

# Uncomment when ready:
# df_eng['login_trend'] = df['logins_last_7d'] - df_eng['expected_weekly_logins']
# df_eng['recent_activity_drop'] = ((df['logins_last_7d'] == 0) & (df['logins_last_30d'] > 0)).astype(int)

In [ ]:
# ============================================
# SELF-CHECK
# ============================================

print(f"login_trend distribution:")
print(f"  Negative (declining): {(df_eng['login_trend'] < 0).mean():.1%}")
print(f"  Positive (growing):   {(df_eng['login_trend'] > 0).mean():.1%}")
print(f"\nrecent_activity_drop: {df_eng['recent_activity_drop'].mean():.1%} of customers")

---

## Part 4: Create Interaction Features

Sometimes combinations of features matter more than features alone.

In [ ]:
# Interaction: New customer with support tickets = high risk
df_eng['new_with_tickets'] = ((df['tenure_months'] < 3) & 
                              (df['support_tickets_last_30d'] > 0)).astype(int)

# Interaction: Low NPS + skipped items = very unhappy
df_eng['unhappy_skipper'] = ((df['nps_score'] < 6) & 
                             (df['items_skipped_last_3_boxes'] > 0)).astype(int)

# Engagement score: combine multiple signals
df_eng['engagement_score'] = (
    df['logins_last_30d'] / (df['logins_last_30d'].max() + 1) +
    df['orders_last_30d'] / (df['orders_last_30d'].max() + 1) -
    df['items_skipped_last_3_boxes'] / 3
)

print("Created interaction features:")
print(f"  new_with_tickets: {df_eng['new_with_tickets'].mean():.1%}")
print(f"  unhappy_skipper: {df_eng['unhappy_skipper'].mean():.1%}")
print(f"  engagement_score range: [{df_eng['engagement_score'].min():.2f}, {df_eng['engagement_score'].max():.2f}]")

---

## Part 5: Verify No Leakage!

Before using new features, check they don't leak future information.

In [ ]:
# Leakage check: our engineered features should not have unrealistic correlation
engineered_features = [
    'orders_per_month', 'support_ticket_rate', 'login_consistency',
    'login_trend', 'recent_activity_drop', 'new_with_tickets',
    'unhappy_skipper', 'engagement_score'
]

print("=== Correlation with Churn ===")
print("(Suspicious if > 0.5 absolute)\n")

for feat in engineered_features:
    if df_eng[feat].notna().any():
        corr = df_eng[feat].corr(df_eng['churn_30d'])
        flag = "⚠️ CHECK THIS" if abs(corr) > 0.5 else "✓"
        print(f"  {feat:25} {corr:+.3f}  {flag}")

---

## Part 6: Train Model with Engineered Features

In [ ]:
# Combined feature set
all_features = raw_features + engineered_features

X_eng = df_eng[all_features].fillna(-1)

# Same train/test split
X_train_eng, X_test_eng, _, _ = train_test_split(X_eng, y, test_size=0.2, random_state=42)

# Train model with engineered features
eng_model = lgb.LGBMClassifier(n_estimators=100, max_depth=5, random_state=42, verbose=-1)
eng_model.fit(X_train_eng, y_train)

eng_auc = roc_auc_score(y_test, eng_model.predict_proba(X_test_eng)[:, 1])

print(f"=== AUC Comparison ===")
print(f"Baseline (raw features only): {baseline_auc:.3f}")
print(f"With engineered features:     {eng_auc:.3f}")
print(f"\nImprovement: {eng_auc - baseline_auc:+.3f}")

In [ ]:
# Which features matter now?
importance_df = pd.DataFrame({
    'feature': all_features,
    'importance': eng_model.feature_importances_
}).sort_values('importance', ascending=True)

plt.figure(figsize=(10, 8))
colors = ['steelblue' if f in raw_features else 'darkgreen' for f in importance_df['feature']]
plt.barh(importance_df['feature'], importance_df['importance'], color=colors)
plt.xlabel('Importance')
plt.title('Feature Importance (blue=raw, green=engineered)')
plt.tight_layout()
plt.show()

In [ ]:
# Precision@500 comparison
k = 500
baseline_probs = baseline_model.predict_proba(X_test_raw)[:, 1]
eng_probs = eng_model.predict_proba(X_test_eng)[:, 1]

baseline_prec = y_test.iloc[np.argsort(baseline_probs)[-k:]].mean()
eng_prec = y_test.iloc[np.argsort(eng_probs)[-k:]].mean()
random_prec = y_test.mean()

print(f"=== Precision@{k} ===")
print(f"Random:     {random_prec:.1%}")
print(f"Baseline:   {baseline_prec:.1%} (lift: {baseline_prec/random_prec:.1f}x)")
print(f"Engineered: {eng_prec:.1%} (lift: {eng_prec/random_prec:.1f}x)")

---

## Part 7: Feature Engineering Recipe

A practical checklist for any tabular ML project.

In [ ]:
recipe = """
┌─────────────────────────────────────────────────────────────────────────────┐
│  FEATURE TYPE        │  PATTERN                    │  EXAMPLE               │
├─────────────────────────────────────────────────────────────────────────────┤
│  Ratios              │  A / (B + eps)              │  orders_per_month      │
│  Changes / Trends    │  recent - historical        │  login_trend           │
│  Binary flags        │  condition.astype(int)      │  recent_activity_drop  │
│  Interactions        │  (condition1 & condition2)  │  new_with_tickets      │
│  Aggregations        │  sum / mean / std of group  │  avg_order_value       │
│  Time-based          │  days_since_X, month, dow   │  days_since_last_order │
└─────────────────────────────────────────────────────────────────────────────┘

LEAKAGE CHECK for every feature:
  "At prediction time, would I have this information?"
  
  ✓ Safe: anything based on past behavior
  ✗ Leakage: anything that depends on future outcome
"""
print(recipe)

---

## 📝 Final Exercise: Explain It

You present your improved model to stakeholders. One asks: "How did you improve it by 3%? Did you use more data?"

Write a 4-5 sentence response explaining feature engineering in non-technical terms.

In [ ]:
# Write your response:

stakeholder_response = """
YOUR RESPONSE HERE
"""

print(stakeholder_response)

---

## ✅ Module 5 Complete!

**What you learned:**
- How to create ratio, change, and interaction features
- Why these often improve models
- How to check for leakage in engineered features
- A practical feature engineering recipe

In [ ]:
# Summary
print("=== Module 5 Summary ===")
print(f"\nFeatures: {len(raw_features)} raw → {len(all_features)} total")
print(f"AUC: {baseline_auc:.3f} → {eng_auc:.3f} ({eng_auc - baseline_auc:+.3f})")
print(f"Precision@500: {baseline_prec:.1%} → {eng_prec:.1%}")
print(f"\nTop engineered feature: {importance_df[importance_df['feature'].isin(engineered_features)].iloc[-1]['feature']}")

**Next:** [Module 6: Evaluating What Matters →](./module_06_evaluation.ipynb)